In [3]:
# prediction of rhabdomyolysis inducing drugs from structure-property relationship
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors  
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
fron sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# read the data
rhabdo_data = pd.read_csv('../rhabdo prediction/rhabdo_dataset.csv')

# calculate the molecular fingerprints
rhabdo_data['Mol'] = [Chem.MolFromSmiles(x) for x in rhabdo_data['smiles']]
rhabdo_data['MorganFP'] = [AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=1024) for x in rhabdo_data['Mol']]
rhabdo_data

[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not removing hydrogen atom without neighbors
[12:36:39] WARNING: not r

,drug,smiles,CID,rhabdo_inducer,Mol,MorganFP
0,Simvastatin,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,54454,1,<rdkit.Chem.rdchem.Mol object at 0x0000024042B...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Cerivastatin,CC(C)C1=C(C(=C(C(=N1)C(C)C)COC)C2=CC=C(C=C2)F)...,446156,1,<rdkit.Chem.rdchem.Mol object at 0x0000024042B...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Atorvastatin,CC(C)C1=C(C(=C(N1CC[C@H](C[C@H](CC(=O)O)O)O)C2...,60823,1,<rdkit.Chem.rdchem.Mol object at 0x0000024042B...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Rosuvastatin,CC(C)C1=NC(=NC(=C1/C=C/[C@H](C[C@H](CC(=O)O)O)...,446157,1,<rdkit.Chem.rdchem.Mol object at 0x0000024042B...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Gemfibrozil,CC1=CC(=C(C=C1)C)OCCCC(C)(C)C(=O)O,3463,1,<rdkit.Chem.rdchem.Mol object at 0x0000024042B...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
...,...,...,...,...,...,...
1276,Technetium TC-99M apcitide,CC(=O)NCSCC(C(=O)NCC(=O)NC(CSCNC(=O)C)C(=NCC(=...,155491107,0,<rdkit.Chem.rdchem.Mol object at 0x0000024042C...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
1277,Technetium ((sup 99m)Tc) Methylenediphosphonate,C(P(=O)([O-])[O-])P(=O)([O-])[O-].O.O.[Tc],155491116,0,<rdkit.Chem.rdchem.Mol object at 0x0000024042C...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1278,Technetium tc-99m bicisate,CCOC(=O)C(C[S-])NCC[N-]C(C[S-])C(=O)OCC.[O-2]....,155491161,0,<rdkit.Chem.rdchem.Mol object at 0x0000024042C...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1279,Technetium TC 99M pyrophosphate,[O-]P(=O)([O-])OP(=O)([O-])[O-].[Tc],155491164,0,<rdkit.Chem.rdchem.Mol object at 0x0000024042C...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
rhabdo_data['MorganFP'][0]